In [1]:
from base import setup, ROBUST_GAMMA
from simpleac import SimPleAC
import ipywidgets as widgets


performance = widgets.FloatSlider(description = "Performance [lbf]",
                                  value = 1300, min = 1, max=2000, step = .1)
wing_weight_pr = widgets.FloatSlider(description = "Wing Weight Percentage",
                                  value = 5, min = 0, max=30, step = .01)
tsfc_pr = widgets.FloatSlider(description = "TSFC Percentage",
                                  value = 15, min = 0, max=30, step = .01)
v_min_pr = widgets.FloatSlider(description = "Takeoff Speed Percentage",
                                  value = 5, min = 0, max=30, step = .01)
range_pr = widgets.FloatSlider(description = "Range Speed Percentage",
                                  value = 15, min = 0, max=30, step = .01)

levers = [performance, wing_weight_pr, tsfc_pr, v_min_pr, range_pr]

from simpleac import SimPleAC
from robust.robust import RobustModel
from gpkit import Variable, units

def constructor():
    gamma = Variable('Gamma', '-', 'Uncertainty bound')
    m = SimPleAC(wing_weight_pr.value, tsfc_pr.value, v_min_pr.value, range_pr.value)
    nominal_sol = m.localsolve(verbosity=0)

    m.append(m["W_f"] <= performance.value*units.lbf)
    m.append(gamma <= 1e30)
    m.cost = 1/gamma

    rm = RobustModel(m, "box", gamma=gamma,
                     twoTerm=False, boyd=False, simpleModel=True,
                     nominalsolve=nominal_sol)
    return rm
    

display(setup(levers, lambda _: {}, constructor, ROBUST_GAMMA))